# Introductory demo of Core Imaging Library (CIL) 

### 3D laboratory micro-CT, cone-beam data of walnut

First import all modules we will need:

In [ ]:
import numpy as np
import os

from cil.io import TXRMDataReader, TIFFWriter
from cil.processors import TransmissionAbsorptionConverter, CentreOfRotationCorrector, Slicer
from cil.plugins.astra import FBP
from cil.utilities.display import show2D, show_geometry

Load the 3D cone-beam projection data of a walnut:

In [ ]:
filename = "/media/newhd/shared/Data/zeiss/walnut/valnut/valnut_2014-03-21_643_28/tomo-A/valnut_tomo-A.txrm"
data = TXRMDataReader(file_name=filename).read()

The data is loaded in as a CIL AcqisitionData object:

In [ ]:
type(data)

We can call print for the data to get some basic information:

In [ ]:
print(data)

Note how labels refer to the different dimensions. We infer that this data set contains 1601 projections each size 1024x1024 pixels.

In addition to the data itself, AcquisionData contains geometric metadata in an AcquisitionGeometry object in the `geometry` field, which can be printed for more detailed information:

In [ ]:
print(data.geometry)

CIL can illustrate the scan setup visually from the AcquisitionData geometry:

In [ ]:
show_geometry(data.geometry)

We can use the dimension labels to extract and display 2D slices of data, such as a single projection:

In [ ]:
show2D(data, slice_list=('angle',800))

From the background value of 1.0 we infer that the data is transmission data (it is known to be already centered and flat field corrected) so we just need to convert to absorption/apply the negative logarithm, which can be done using a CIL processor, which will handle small/large outliers:

In [ ]:
data = TransmissionAbsorptionConverter()(data)

We again take a look at a slice of the data, now a vertical one to see the central slice sinogram after negative logarithm:

In [ ]:
show2D(data, slice_list=('vertical',512))

CIL supports different back-ends for which data order conventions may differ. Here we use the FBP algorithm from the ASTRA Toolbox, which requires us to permute the data array into the right order:

In [ ]:
data.reorder(order='astra')

The data is now ready for reconstruction. To set up the FBP algorithm we must specify the size/geometry of the reconstruction volume. Here we use the default one:

In [ ]:
ig = data.geometry.get_ImageGeometry()

We can then create the FBP algorithm (really FDK since 3D cone-beam) from ASTRA running on the GPU and reconstruct the data:

In [ ]:
fbp =  FBP(ig, data.geometry, "gpu")
recon = fbp(data)

We show the central (vertical) slice:

In [ ]:
show2D(recon, slice_list=('vertical',512), fix_range=(-0.01,0.06))

And a horizontal (horizontal_x) slice as well:

In [ ]:
show2D(recon, slice_list=('horizontal_x',512), fix_range=(-0.01,0.06))

We can save the reconstructed volume to disk for example as a stack of TIFFs:

In [ ]:
# TIFFWriter(data=recon, file_name="/media/newhd/shared/Data/out").write()